In [ ]:
!pip install langchain
!pip install pypdf
!pip install unstructured
!pip install sentence_transformers
!pip install pinecone-client
!pip install llama-cpp-python
!pip install huggingface_hub

In [18]:
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer
from langchain.chains.question_answering import load_qa_chain
import pinecone
import os

In [32]:
loader = PyPDFLoader("/content/2308.12261v1.pdf")

In [20]:
loader = PyPDFLoader("/content/2308.13418v1.pdf")

In [ ]:
data = loader.load()

data

In [34]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)


docs=text_splitter.split_documents(data)


len(docs)


94

In [35]:
docs[0]

Document(page_content='PROMPT 2MODEL :\nGenerating Deployable Models from Natural Language Instructions\nVijay Viswanathan1∗, Chenyang Zhao1,2∗,\nAmanda Bertsch1, Tongshuang Wu1, Graham Neubig1\n1Carnegie Mellon University,2Tsinghua University\nAbstract\nLarge language models (LLMs) enable system\nbuilders today to create competent NLP sys-\ntems through prompting, where they only need\nto describe the task in natural language and\nprovide a few examples. However, in other\nways, LLMs are a step backward from tradi-', metadata={'source': '/content/2308.12261v1.pdf', 'page': 0})

In [36]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_eLonpUwvGKPtZGTywgjBx"
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'd2d6c3d-9d50-37a8a0697ed3')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [37]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [38]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)
index_name = "multipdfchat"

In [39]:
docsearch=Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)

In [42]:
query="what is Nougat"


docs=docsearch.similarity_search(query)


docs

[Document(page_content='3https://github.com/facebookresearch/nougatarXiv:2308.13418v1  [cs.LG]  25 Aug 2023', metadata={}),
 Document(page_content='Explore the examples in this section on the project page: https://facebookresearch.github.io/nougat.\n11https://archive.org/\n12https://www.gutenberg.org/\n13', metadata={}),
 Document(page_content='Nougat base (350M∗)All 0.071 89.1 93.0 93.5 92.8 93.1\nTables 0.211 69.7 79.1 75.4 80.7 78.0\nPlain text 0.058 91.2 94.6 96.2 95.3 95.7\nMath 0.128 56.9 75.4 76.5 76.6 76.5\nTable 1: Results on arXiv test set. PDF is the text embedded in the PDF file. The modality “All” refers to the output\ntext without any splitting.∗Number of parameters.\n5.2 Text modalities\nIn a scientific research article, there are three distinct types of text: 1) plain text, which comprises the majority of the', metadata={}),
 Document(page_content='Nougat Blecher et al.\nMethod Modality Edit distance ↓BLEU ↑METEOR ↑Precision ↑Recall ↑F1↑\nPDF All 0.255 65.8 82.1 77.1 81

In [41]:
from langchain.llms import HuggingFaceHub


llm=HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.1, "max_length":512})


chain=load_qa_chain(llm, chain_type="stuff")


query="Prompting LLMs can be expensive as"
docs=docsearch.similarity_search(query)


chain.run(input_documents=docs, question=query)

'they require either a significant amount of com- puting or access to commercial APIs'